In [0]:
import numpy as np
from numpy import genfromtxt
from sklearn import svm
from sklearn.metrics import accuracy_score
import random
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import numpy.linalg as la
import scipy.io as sio
import pickle
from cvxopt import matrix, solvers
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


We implement all of the methods using a V-matrix framework. Note that for all previous methods, the V-matrix is diagonal. This is because all previous methods assign an importance or weight to each point, whose lose is signified by the diagonal. Our contribution is to use Vapnik's framework, which utilizes the entire matrix. Kleip is run on the original Matlab code and the weights are imported.

In [0]:
def V_matrix_Huang(T,X,n_t,sig=0.1):
  l = X.shape[0]
  m = T.shape[0]
  V = np.identity(l)
  P = matrix(gram_matrix(X,sig))
  K = Kernel.gaussian(sig)
  sum_T = [0 for i in range(l)]
  for i in range(l):
    for t in range(m):
      sum_T[i]+=K(T[t],X[i])
    sum_T[i] = -1*l/m*sum_T[i]  
  q = matrix(sum_T)
  G = [[0]*l for i in range(2*l+2)]
  for i in range(l):
    G[i][i] = 1
    G[i+1][i] = -1
  for j in range(l):
    G[2*l][j] = 1
    G[2*l+1][j] = -1
 
  G = matrix(np.array(G),tc ='d')
  B = 1000
  eps = 1-1/np.sqrt(l)
  h = [0 for i in range(2*l+2)]
  for i in range(l):
    h[i] = B
  h[2*l] = l+l*eps
  h[2*l+1] = l-l*eps   
  h = matrix(h)
  solvers.options['show_progress'] = False
  sol = solvers.qp(P,q,G,h)
  solution = np.array(sol['x'])
  diagonal = [solution[i][0] for i in range(l)]
  V = np.diag(diagonal)
  return V 

def V_matrix_new(T,X,n_t):
  l = X.shape[0]
  V = np.identity(l) # our V
  for i in range(l):
    for j in range(l):
      V[i,j] = 0
      for t in T:
        u = 1
        for d in range(T.shape[1]):
          if t[d] < np.maximum(X[i,d],X[j,d]):
            u = 0
            break
        V[i,j]+=u  
      V[i,j] = V[i,j]/n_t 
  #P,Q,R = la.svd(V)
  #Q_hat = np.diag([Q[i] + .01 for i in range(l)])
  #V = np.matmul(np.matmul(P,Q_hat),R) #Regularizing V in high dimensions     
  return V
def V_matrix_new_2(T,X,n_t):
  l = X.shape[0]
  V = np.identity(l) # our V
  for i in range(l):
    for j in range(l):
      V[i,j] = 1
      for d in range(T.shape[1]):
        u = 0
        for t in T:
          if t[d] >= np.maximum(X[i,d],X[j,d]):
            u+= 1
        u = u/n_t
        V[i,j]*=u  
      #V[i,j] = V[i,j]/n_t 
  #P,Q,R = la.svd(V)
  #Q_hat = np.diag([Q[i] + .01 for i in range(l)])
  #V = np.matmul(np.matmul(P,Q_hat),R) #Regularizing V in high dimensions     
  return V

def V_matrix_LDA(T, X, n_t, lda):
  X_r = lda.transform(X)
  T_r = lda.transform(T)

  l = X.shape[0]
  V = np.identity(l) # our V
  for i in range(l):
    for j in range(l):
      V[i,j] = 0
      for t in T_r:
        u = 1
        for d in range(T_r.shape[1]):
          if t[d] < np.maximum(X_r[i,d],X_r[j,d]):
            u = 0
            break
        V[i,j]+=u    
      V[i,j] = V[i,j]/n_t 
  #P,Q,R = la.svd(V)
  #Q_hat = np.diag([Q[i] for i in range(l)])
  #V = np.matmul(np.matmul(P,Q_hat),R) #Regularizing V in high dimensions     
  return V



def V_matrix_add(T,X,n_t):
  l = X.shape[0]
  V = np.identity(l) # our V
  for i in range(l):
    for j in range(l):
      V[i,j] = 0
      for t in T:
        u = 0
        for d in range(T.shape[1]):
          if t[d] >= np.maximum(X[i,d],X[j,d]):
            u += 1
        V[i,j]+=u/T.shape[1]    
      V[i,j] = V[i,j]/n_t 
  #P,Q,R = la.svd(V)
  #Q_hat = np.diag([Q[i] for i in range(l)])
  #V = np.matmul(np.matmul(P,Q_hat),R) #Regularizing V in high dimensions     
  return V

def Vapnik(X):
  l = X.shape[0]
  V = np.array([[1]*l for i in range(l)]) 
  for i in range(l):
    for j in range(l):
      for d in range(X.shape[1]):
        V[i,j]+= 1-np.maximum(X[i,d],X[j,d])
  P,Q,R = la.svd(V)
  #Q_hat = np.diag([Q[i]+0.001 for i in range(l)])
  #V = np.matmul(np.matmul(P,Q_hat),R) #Regularizing V in high dimensions 
  return V

def unif_kernel(x,X,h):
  n = X.shape[0]
  val = 0
  for i in range(n):
    if la.norm((x-X[i])/h,1) <=1:
      val+= 0.5  
  q = val/(n*h)
  return q

def gauss_kernel(x,X,h):
  n = X.shape[0]
  val = 0
  for i in range(n):
    val+=1/(np.sqrt(2*np.pi))*(np.exp(-la.norm((x-X[i])/h)**2/2))
  q = val/(n*h)
  return q

def V_matrix_Shimodaira(T,X,n_t,h):
  l = X.shape[0]
  V = np.identity(l)
  for i in range(l):
    l_1 = gauss_kernel(X[i],T,h)
    l_0 = gauss_kernel(X[i],X,h)
    V[i,i] = l_1/l_0
  return V

def V_matrix_Sugiyama(T,X,n_t,h,tau):
  l = X.shape[0]
  V = np.identity(l)
  for i in range(l):
    l_1 = gauss_kernel(X[i],T,h)
    l_0 = gauss_kernel(X[i],X,h)
    V[i,i] = (l_1/l_0)**tau
  return V

def V_matrix_Makoto(T,X,n_t,h,a):
  l = X.shape[0]
  V = np.identity(l)
  for i in range(l):
    l_1 = gauss_kernel(X[i],T,h)
    l_0 = gauss_kernel(X[i],X,h)
    V[i,i] = l_1/(a*l_0+(1-a)*l_1)
  return V  

def test_methods_review(T,X,Y,Y_test,V):
  U = gram_matrix(X)# get the gram matrix for the given training set X
  g = 0.1  
  A, c = A_estimate(V,U,Y,g)
  p = []
  p_exp = Y_test
  for t in range(T.shape[0]):
    U_l = style_K(X,T[t]) #U_l = # kernel for the test set
    p.append(p_estimate(A,c,U_l))
  p = np.array(p)
  e = la.norm(p_exp-p)**2/T.shape[0]
  return e


#Estimate of A and c
def A_estimate(V,U,Y,g):
  l = V.shape[0]
  I = np.identity(l)
  O = np.ones((l,1))
  a_1 = np.matmul(V,U)
  a_2 = np.matmul(V,Y.reshape(l,1))
  a_3 = np.matmul(V,O)
  W = np.linalg.inv(a_1+g*I) # g:regularization constant
  A_b = np.matmul(W,a_2).reshape(l,1) # from equation 14
  A_c = np.matmul(W,a_3).reshape(l,1) # from equation 14
  c_1 = np.matmul(O.transpose(),np.matmul(a_1,A_b)-a_2) # numerator in equation 14
  c_2 = np.matmul(O.transpose(),np.matmul(a_1,A_c)-a_3)# denominator in equation 14
  c = c_1/c_2 # from equation 14
  A = A_b - c[0,0]*A_c
  return A,c

#Conditional probability estimate for a given x
def p_estimate(A,c,U_l):
  p = np.matmul(A.transpose(),U_l)+c
  if p[0,0] > 0.5:
    return 1
  else:
    return 0  

def gram_matrix(X,sig = 0.1):
   l = X.shape[0]
   U = np.zeros((l,l))
   K = Kernel.gaussian(sig)         
   for i in range(l):
     for j in range(l):
       U[i,j] = K(X[i],X[j])
   return U

def style_K(X,t):
   l = X.shape[0]
   U_l = np.zeros((l,1))
   K = Kernel.gaussian(.1)
   for i in range(l):
     U_l[i] = K(t,X[i])    
   return U_l  

class Kernel(object):
    """Implements list of kernels from
    http://en.wikipedia.org/wiki/Support_vector_machine
    """
    @staticmethod
    def linear():
        def f(x, y):
            return np.inner(x, y)
        return f

    @staticmethod
    def gaussian(sigma):
        def f(x, y):
            #exponent = -np.sqrt(sigma*la.norm(x-y)**2)
            exponent = -sigma*la.norm(x-y)**2
            return np.exp(exponent)
        return f

    @staticmethod
    def _polykernel(dimension, offset):
        def f(x, y):
            return (offset + np.dot(x, y)) ** dimension
        return f

    @staticmethod
    def inhomogenous_polynomial(dimension):
        return Kernel._polykernel(dimension=dimension, offset=1.0)

    @staticmethod
    def homogenous_polynomial(dimension):
        return Kernel._polykernel(dimension=dimension, offset=0.0)

    @staticmethod
    def hyperbolic_tangent(kappa, c):
        def f(x, y):
            return np.tanh(kappa * np.dot(x, y) + c)
        return f

def instance(X,T,Y,Y_test, kliep_weights, ulsif_weights,sig = 0.1):
  l = X.shape[0]
  n_t = T.shape[0]
  h = 2.0
  tau = 0.5
  alpha = 0.5 
  
  # Trim T to 500 datapoints
  trim_length = 500
  T_sub = T
  V_1 = np.identity(l)                      # Unweighted
  #V_2 = Vapnik(X)                           # Vapnik
  V_3= V_matrix_add(T_sub,X,n_t)            # Proposed
  #V_4 = V_matrix_Shimodaira(T_sub,X,n_t,h)  # Shimodaira 
  #V_5 = V_matrix_Sugiyama(T_sub,X,n_t,h,tau)# Sugiyama
  V_6 = V_matrix_Makoto(T_sub,X,n_t,h,alpha)# Makoto
  V_7 = V_matrix_Huang(T_sub,X,n_t,sig)         # Huang
  V_8 = np.diag(kliep_weights)              # KLIEP
  V_9 = np.diag(ulsif_weights)              # ULSIF

  error_1 = test_methods_review(T,X,Y,Y_test,V_1)
  print("Error Unweighted: {}".format(error_1))
  #error_2 = test_methods_review(T,X,Y,Y_test,V_2)
  #print("Error Vapnik: {}".format(error_2))
  error_3 = test_methods_review(T,X,Y,Y_test,V_3)
  print("Error Proposed: {}".format(error_3))
  #error_4 = test_methods_review(T,X,Y,Y_test,V_4)
  #print("Error Shimodaira: {}".format(error_4))
  #error_5 = test_methods_review(T,X,Y,Y_test,V_5)
  #print("Error Sugiyama: {}".format(error_5))
  error_6 = test_methods_review(T,X,Y,Y_test,V_6)
  print("Error Makoto: {}".format(error_6))
  error_7 = test_methods_review(T,X,Y,Y_test,V_7)
  print("Error Huang: {}".format(error_7))
  error_8 = test_methods_review(T,X,Y,Y_test,V_8)
  print("Error KLIEP: {}".format(error_8))
  error_9 = test_methods_review(T,X,Y,Y_test,V_9)
  print("Error ULSIF: {}".format(error_9))
  return [error_1,error_3,error_4,error_5,error_6,error_7, error_8, error_9]
  #return [error_1,error_7]

Load previously generated datasets:

In [0]:
dsets_read = open('datasets.p', 'rb')
dataset_info = pickle.load(dsets_read)
matlab = sio.loadmat('matlab_twonorm.mat')
datasets = [('twonorm', dataset_info['twonorm_data'], dataset_info['train_inds_twonorm'], dataset_info['test_inds_twonorm'], matlab['kliep_twonorm'], matlab['ulsif_twonorm'])]

In [0]:
for name, data, train_inds, test_inds, kliep, ulsif in datasets:
  errs = [[] for i in range(1)]
  trial = 0
  training_sets = []
  for trial in range(len(train_inds)):
    tr_inds = train_inds[trial]
    te_inds = test_inds[trial]#np.array(list(set(range(len(data))) - set(tr_inds)))
    train = data[tr_inds, :]
    test = data[te_inds, :]
    f = train.shape[1] 
    X_train = train[:, :-1]
    y_train = train[:, -1]
    print("Percentage of y=1: {}".format(sum(y_train)))

    X_test = test[:, :-1]
    y_test = test[:, -1]
    sigmas = [10*(1+i) for i in range(1)]
    for j,sig in enumerate(sigmas):
      print(j)
      errors = instance(X_train, X_test, y_train, y_test, kliep[trial], ulsif[trial],sig)
      if errors[0] > 0:
        errors = errors / errors[0]
        print(errors)
        errs[j].append(errors)
      avg_errors = np.mean(errs[j], axis=0)
      std_errors = np.std(errs[j], axis = 0)
      print("CUMULATIVE\n{}\n{}\n{}".format(name, avg_errors, std_errors))

Percentage of y=1: 49.0
0
Error Unweighted: 0.054
Error Huang: 0.04599999999999999
[1.         0.85185185]
CUMULATIVE
twonorm
[1.         0.85185185]
[0. 0.]
Percentage of y=1: 56.0
0
Error Unweighted: 0.038000000000000006
Error Huang: 0.034
[1.         0.89473684]
CUMULATIVE
twonorm
[1.         0.87329435]
[0.        0.0214425]
Percentage of y=1: 47.0
0
Error Unweighted: 0.032
Error Huang: 0.032
[1. 1.]
CUMULATIVE
twonorm
[1.         0.91552956]
[0.         0.06224265]
Percentage of y=1: 50.0
0
Error Unweighted: 0.020000000000000004
Error Huang: 0.020000000000000004
[1. 1.]
CUMULATIVE
twonorm
[1.         0.93664717]
[0.         0.06514193]
Percentage of y=1: 50.0
0
Error Unweighted: 0.028
Error Huang: 0.022
[1.         0.78571429]
CUMULATIVE
twonorm
[1.        0.9064606]
[0.         0.08390289]
Percentage of y=1: 42.0
0
Error Unweighted: 0.03599999999999999
Error Huang: 0.034
[1.         0.94444444]
CUMULATIVE
twonorm
[1.         0.91279124]
[0.         0.07788965]
Percentage of y=1: 

In [0]:
random.sample(range(10),5)

[5, 3, 4, 6, 1]